In [1]:
import sys
# If you want to specific the package address, uncomment the lines below.
# open3SPN2_HOME = '/Users/weilu/open3spn2/'
# openAWSEM_HOME = '/Users/weilu/openmmawsem/'
# sys.path.insert(0,open3SPN2_HOME)
# sys.path.insert(0,openAWSEM_HOME)
import open3SPN2
import ffAWSEM
import os
from simtk.openmm.app import PDBFile
from functools import partial

In [2]:
#Fix the system (adds missing atoms)
fix=open3SPN2.fixPDB("1lmb.pdb")
# uncommented two line below to generate the single memory.
# PDBFile.writeFile(fix.topology, fix.positions, open("fixed.pdb", 'w'))
# os.system(f"python {openAWSEM_HOME}/create_single_memory_from_pdb.py -p fixed.pdb")

In [4]:
#Create a table containing both the proteins and the DNA
complex_table=open3SPN2.pdb2table(fix)

In [5]:
#Coarse Grain the system
dna_atoms=open3SPN2.DNA.CoarseGrain(complex_table)
protein_atoms=ffAWSEM.Protein.CoarseGrain(complex_table)

In [6]:
#Merge the models
import pandas
Coarse=pandas.concat([protein_atoms,dna_atoms],sort=False)
Coarse.index=range(len(Coarse))
Coarse.serial=list(Coarse.index)

In [7]:
#Save the protein_sequence
from Bio.PDB.Polypeptide import three_to_one
_AWSEMresidues=['IPR','IGL','NGP']
protein_data=Coarse[Coarse.resname.isin(_AWSEMresidues)].copy()
resix = (protein_data.chainID + '_' + protein_data.resSeq.astype(str))
res_unique = resix.unique()
protein_data['resID'] = resix.replace(dict(zip(res_unique, range(len(res_unique)))))
protein_sequence=[r.iloc[0]['real_resname'] for i, r in protein_data.groupby('resID')]
protein_sequence_one = [three_to_one(a) for a in protein_sequence]

with open('protein.seq','w+') as ps:
    ps.write(''.join(protein_sequence_one))

In [8]:
# Create a merged PDB
def writePDB(atoms,pdb_file):
    with open(pdb_file, 'w+') as pdb:
        for i, atom in atoms.iterrows():
            pdb_line = f'{atom.recname:<6}{atom.serial:>5} {atom["name"]:^4}{atom.altLoc:1}'+\
                       f'{atom.resname:<3} {atom.chainID:1}{atom.resSeq:>4}{atom.iCode:1}   '+\
                       f'{atom.x:>8.3f}{atom.y:>8.3f}{atom.z:>8.3f}' +\
                       f'{atom.occupancy:>6.2f}{atom.occupancy:>6.2f}'+' ' * 10 +\
                       f'{atom.element:>2}{atom.charge:>2}'
            assert len(pdb_line) == 80, f'An item in the atom table is longer than expected ({len(pdb_line)})\n{pdb_line}'
            pdb.write(pdb_line + '\n')
writePDB(Coarse,'clean.pdb')

In [9]:
#Create the merged system
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(ffAWSEM.xml,open3SPN2.xml)
s=forcefield.createSystem(top)

In [10]:
dna=open3SPN2.DNA.fromCoarsePDB('clean.pdb')
with open('protein.seq') as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence_one)
dna.periodic=False
protein.periodic=False

In [11]:
ffAWSEM.copy_parameter_files()

In [12]:
#Clear Forces from the system
keepCMMotionRemover=True
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'
forces={}
for i in range(s.getNumForces()):
    force = s.getForce(i)
    force_name="CMMotionRemover"

#Add 3SPN2 forces
for force_name in open3SPN2.forces:
    print(force_name)
    force = open3SPN2.forces[force_name](dna)
    if force_name in ['BasePair','CrossStacking']:
        force.addForce(s)
    else:
        s.addForce(force)
    forces.update({force_name: force})

#Add AWSEM forces
openAWSEMforces = dict(Connectivity=ffAWSEM.functionTerms.basicTerms.con_term,
                       Chain=ffAWSEM.functionTerms.basicTerms.chain_term,
                       Chi=ffAWSEM.functionTerms.basicTerms.chi_term,
                       Excl=ffAWSEM.functionTerms.basicTerms.excl_term,
                       rama=ffAWSEM.functionTerms.basicTerms.rama_term,
                       rama_pro=ffAWSEM.functionTerms.basicTerms.rama_proline_term,
                       #rama_ss=ffAWSEM.functionTerms.basicTerms.rama_ssweight_term,
                       contact=ffAWSEM.functionTerms.contactTerms.contact_term,
                       frag  = partial(ffAWSEM.functionTerms.templateTerms.fragment_memory_term, frag_file_list_file="./single_frags.mem", npy_frag_table="./single_frags.npy", UseSavedFragTable=False),
                       beta1 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_1,
                       beta2 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_2,
                       beta3 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_3,
                       pap1 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_1,
                       pap2 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_2,
                      )
protein.setup_virtual_sites(s)
for force_name in openAWSEMforces:
    print(force_name)
    if force_name in ['contact']:
        force = openAWSEMforces[force_name](protein, withExclusion=False,periodic=False)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    elif force_name in ['Excl']:
        force = openAWSEMforces[force_name](protein)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    else:
        force = openAWSEMforces[force_name](protein)
    s.addForce(force)
    forces.update({force_name: force})

#Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print(force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna,protein)
    s.addForce(force)
    forces.update({force_name: force})

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
Connectivity
Chain
Chi
Excl
1205
1844
rama
rama_pro
contact
Number of atom:  1171 Number of residue:  179
Contact cutoff  1.0 nm
NonbondedMethod:  1
0
639
frag
Loading Fragment files(Gro files)
Saving fragment table as npy file to speed up future calculation.
All gro files information have been stored in the ./single_frags.npy.             
You might want to set the 'UseSavedFragTable'=True to speed up the loading next time.             
But be sure to remove the .npy file if you modify the .mem file. otherwise it will keep using the old frag memeory.
beta1
beta_1 term ON
beta2
beta_2 term ON
beta3
beta_3 term ON
pap1
pap_1 term ON
No ssweight given, assume all zero
pap2
pap_2 term ON
No ssweight given, assume all zero
ExclusionProteinDNA
ElectrostaticsProteinDNA


In [13]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin
#platform_name='CUDA'
platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-4745.88330078125


In [14]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -4745.883301 kJ/mol
Bond 0.0 kJ/mol
Angle 0.0 kJ/mol
Stacking 203.567078 kJ/mol
Dihedral -503.999969 kJ/mol
BasePair -284.232269 kJ/mol
CrossStacking -47.586151 kJ/mol
Exclusion 23.991549 kJ/mol
Electrostatics 23.268295 kJ/mol
Connectivity 1899.296631 kJ/mol
Chain 1899.296631 kJ/mol
Chi 1899.296631 kJ/mol
Excl 1899.296753 kJ/mol
rama -1363.522949 kJ/mol
rama_pro -1363.522949 kJ/mol
contact -1041.547729 kJ/mol
frag -3639.894531 kJ/mol
beta1 -300.796753 kJ/mol
beta2 -300.796753 kJ/mol
beta3 -300.796753 kJ/mol
pap1 0.0 kJ/mol
pap2 0.0 kJ/mol
ExclusionProteinDNA 296.033447 kJ/mol
ElectrostaticsProteinDNA -10.45981 kJ/mol


In [15]:
#Add simulation reporters
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [16]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-5689.087890625,308.9777190301704
2000,3.999999999999781,-5751.890625,312.6522537930845
3000,5.999999999999561,-5731.673828125,297.4589744539855
4000,7.999999999999341,-5415.681640625,307.6215130085438
5000,10.000000000000009,-5415.34765625,302.8099185308025
6000,12.000000000000677,-5650.13818359375,302.4086831431342
7000,14.000000000001345,-5732.2607421875,307.71066925628355
8000,16.00000000000201,-5663.80224609375,295.47726065103393
9000,18.000000000000902,-5708.4150390625,288.1171210367876
10000,19.999999999999794,-5670.01611328125,313.12966690178314
